<a href="https://colab.research.google.com/github/AlexElyg/RAG_BZ/blob/main/project_BZ_3_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Установка библиотек и загрузка модели

In [1]:
!pip install langchain==0.3.26 transformers==4.53.0 sentence_transformers==5.0.0 tiktoken==0.9.0 vllm==0.9.1 -q unstructured
!pip install  langchain-community==0.3.27  -q
!pip install faiss-gpu-cu12==1.11.0 -q
!pip install numpy==2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 27.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 122.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.6/394.6 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 21.1 MB/s eta 0:00:00
   ━━━━

In [ ]:
# @title проверить версии библиотек
#!pip list # все версии установленных библиотек
import langchain
import faiss
import transformers
import vllm
import sentence_transformers
import tiktoken
import numpy
print(f"Langchain version: {langchain.__version__}")
print(f"Faiss version: {faiss.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"vllm version: {vllm.__version__}")
print(f"numpy version: {numpy.__version__}")
print(f"sentence_transformers version: {sentence_transformers.__version__}")
print(f"tiktoken version: {tiktoken.__version__}")

In [2]:
!pip install -U faiss-gpu-cu12 # пока не починят numpy

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.0
    Uninstalling numpy-2.0.0:
      Successfully uninstalled numpy-2.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
import os
import re
import json
import torch
from io import BytesIO
from transformers import AutoTokenizer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from IPython.display import HTML, display
from google.colab import files
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from vllm import LLM, SamplingParams
import textwrap
import logging
from typing import Dict

INFO 07-21 08:01:48 [__init__.py:244] Automatically detected platform cuda.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!wget https://huggingface.co/yandex/YandexGPT-5-Lite-8B-instruct-GGUF/resolve/main/YandexGPT-5-Lite-8B-instruct-Q4_K_M.gguf

--2025-07-21 06:07:26--  https://huggingface.co/yandex/YandexGPT-5-Lite-8B-instruct-GGUF/resolve/main/YandexGPT-5-Lite-8B-instruct-Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 3.163.189.90, 3.163.189.37, 3.163.189.74, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/0c/6d/0c6d537f3e2f06156d8d15715d9699899a9cb6669c7feff0f83af50da39ae641/d9ff5b826f20fbcc2f898f9f2349ac21241579f8fbb79cd32148a333623ba228?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27YandexGPT-5-Lite-8B-instruct-Q4_K_M.gguf%3B+filename%3D%22YandexGPT-5-Lite-8B-instruct-Q4_K_M.gguf%22%3B&Expires=1753081646&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MzA4MTY0Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzBjLzZkLzBjNmQ1MzdmM2UyZjA2MTU2ZDhkMTU3MTVkOTY5OTg5OWE5Y2I2NjY5YzdmZWZmMGY4M2FmNTBkYTM5YWU2NDEvZDlmZjViODI2Zj

# Создание векторной базы и конвертер docx --> MD

In [ ]:
# @title создание локальных папок

#new_folder_name = '/content/annotation'
#os.makedirs(new_folder_name)

# Определяем родительскую директорию
parent_directory = '/content/'
# Список имен новых папок
folder_names = ['output_MD_files',
                'docx_files',
                'create_index_base',
                'BZ',
                'doc'
                ]
# Проверяем существование родительской директории и создаем её, если нужно
if not os.path.exists(parent_directory):
    os.makedirs(parent_directory)
# Создаем каждую папку внутри родительской директории
for folder in folder_names:
    new_path = os.path.join(parent_directory, folder)
    if not os.path.exists(new_path):
        os.makedirs(new_path)
        print(f"Папка {folder} создана.")
    else:
        print(f"Папка {folder} уже существует.")

In [ ]:
# @title создание папок для google disk
parent_directory = '/content/drive/MyDrive/meta_base_faiss/project_BZ'
# Список имен новых папок
folder_names = ['overall',
                'ChS',
                'SVO'
                ]
# Проверяем существование родительской директории и создаем её, если нужно
if not os.path.exists(parent_directory):
    os.makedirs(parent_directory)
# Создаем каждую папку внутри родительской директории
for folder in folder_names:
    new_path = os.path.join(parent_directory, folder)
    if not os.path.exists(new_path):
        os.makedirs(new_path)
        print(f"Папка {folder} создана.")
    else:
        print(f"Папка {folder} уже существует.")

## если нужно конвертировать с meta data

In [ ]:
!wget https://github.com/jgm/pandoc/releases/download/3.7.0.2/pandoc-3.7.0.2-1-amd64.deb
!dpkg -i pandoc-3.7.0.2-1-amd64.deb
# Для работы с изображениями (если они есть в DOCX)
!apt-get install librsvg2-bin -y  # для SVG (опционально)
!apt-get install imagemagick -y   # для других изображений (опционально)
!pip install -U pypandoc -q     # Установка Python-обёртки

In [ ]:
# @title Конвертер с метаданными для документов
import pypandoc
import re
import json
import hashlib
from pathlib import Path

# Создаем Lua-фильтр
with open('remove_links.lua', 'w') as f:
    f.write("""function Link(el) return el.content end""")

def generate_document_id(doc_name):
    """Генерация уникального ID на основе названия файла."""
    return hashlib.md5(doc_name.encode()).hexdigest()[:8]

def add_document_metadata(text, doc_name, doc_id):
    """Добавляет метаданные в начало текста."""
    metadata = f"# Документ: {doc_name}\n**ID:** `{doc_id}`\n\n---\n\n"
    return metadata + text

def save_metadata_to_json(doc_name, doc_id, output_path, metadata_file="/content/output_MD_files/metadata.json"):
    """Сохраняет метаданные в JSON."""
    metadata = {
        "doc_id": doc_id,
        "doc_name": doc_name,
        "source_path": str(output_path)
    }

    # Загружаем существующие метаданные (если есть)
    existing_data = []
    if Path(metadata_file).exists():
        with open(metadata_file, "r", encoding="utf-8") as f:
            existing_data = json.load(f)

    # Добавляем новые метаданные
    existing_data.append(metadata)

    # Сохраняем обновлённые данные
    with open(metadata_file, "w", encoding="utf-8") as f:
        json.dump(existing_data, f, indent=2, ensure_ascii=False)

def prepare_for_rag(docx_path, output_path):
    """Конвертирует DOCX в чистый текст с метаданными."""
    doc_name = Path(docx_path).stem  # Название без расширения (СТО, КСУП, ...)
    doc_id = generate_document_id(doc_name)

    # 1. Конвертация DOCX → Markdown
    try:
        pypandoc.convert_file(
            docx_path,
            'md',
            outputfile='/content/output_MD_files/temp.md',
            format='docx',
            extra_args=['--lua-filter=remove_links.lua',
                       '--wrap=none',
                       "--strip-comments",
                       "--markdown-headings=atx"]
        )
    except Exception as e:
        print(f"Conversion error: {str(e)}")
        raise

    # 2. Чтение и очистка текста
    with open('/content/output_MD_files/temp.md', 'r', encoding='utf-8-sig') as f:
        text = f.read()

    # 3. Очистка и форматирование (похоже на предыдущий код)
    def clean_text(text):
        # Удаление разделителей (2.1 и 2.2)
        text = re.sub(r'^.*={3,}.*$\n?', '', text, flags=re.MULTILINE)  # Удаляет строки с ====
        text = re.sub(r'^.*\.{3,}.*$\n?', '', text, flags=re.MULTILINE) # Удаляет строки с ....

        # Удаление пустых скобок и разделителей
        text = re.sub(r'\[\s*\]', '', text)           # Пустые квадратные скобки
        text = re.sub(r'-{3,}', '\n', text)           # Горизонтальные линии
        text = re.sub(r'\$\$.*?\$\$', '', text)       # Математические формулы
        text = re.sub(r'\s*\n\s*\n[\s\n]*', '\n\n', text) # Лишние пустые строки (2.3)

        # Удаление изображений и пустого форматирования
        text = re.sub(r'!\[.*?\]\(.*?\)', '', text)
        text = re.sub(r'\{\s*width=[^}]*\}', '', text)

        # Удаление якорей и метаданных
        text = re.sub(r'(\{#)?Par\d+.*?\}', '', text)

        # Удаление строк, состоящих только из спецсимволов или цифр
        text = re.sub(r'^[^a-zA-Zа-яА-ЯЁё0-9]*$', '', text, flags=re.MULTILINE)
        # Удаление лишних пробелов по краям строк
        text = '\n'.join([line.strip() for line in text.split('\n') if line.strip()])
        # Нормализация пробелов и переносов
        text = re.sub(r'\n{3,}', '\n\n', text)
        text = re.sub(r'[ \t]{2,}', ' ', text)

        return text.strip()

    cleaned_text = clean_text(text)
    marked_text = add_document_metadata(cleaned_text, doc_name, doc_id)

    # 4. Сохранение результата
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(marked_text)

    print(f"Файл с метаданными сохранён в {output_path}")
    save_metadata_to_json(doc_name, doc_id, output_path)
    return marked_text

# Пример использования
rag_text = prepare_for_rag(
    docx_path='/content/docx_files/Правила технической эксплуатации железных дорог в РФ.docx',
    output_path='/content/doc/Правила технической эксплуатации железных дорог в РФ.md'
)


## создание базы с метаданными

In [ ]:
!pip install unstructured

In [ ]:
# @title Модифицированный код для работы с метаданными СТАРЫЙ

from langchain_community.document_loaders import DirectoryLoader, TextLoader
#new_folder_name = '/content/create_index_base'
#os.makedirs(new_folder_name)

# Загрузка документов с парсингом метаданных
def load_documents(folder_path):
    def parse_metadata(content, file_path):
        """Парсер метаданных с использованием имени файла как резервного варианта"""
        # Получаем имя файла без расширения
        file_name = os.path.splitext(os.path.basename(file_path))[0]

        metadata = {
            "source": file_path,
            "doc_name": file_name,  # По умолчанию используем имя файла
            "doc_id": "N/A"
        }

        # Варианты поиска ID
        id_patterns = [
            r'ID:\s*`?([a-zA-Z0-9]+)`?',
            r'Идентификатор:\s*(\w+)',
            r'Document ID:\s*(\w+)'
        ]

        lines = [line.strip() for line in content.split('\n') if line.strip()]

        # Поиск названия в содержимом (если отличается от имени файла)
        name_patterns = [
            r'^#\s*Документ:\s*(.+?)\s*$',
            r'^#\s*(.+?)\s*\nID:',
            r'^#\s*(.+?)\s*$'
        ]

        for line in lines[:5]:
            for pattern in name_patterns:
                match = re.search(pattern, line, re.IGNORECASE)
                if match and match.group(1).strip().lower() != file_name.lower():
                    metadata["doc_name"] = match.group(1).strip()
                    break

        # Поиск ID
        for line in lines[:10]:
            for pattern in id_patterns:
                match = re.search(pattern, line, re.IGNORECASE)
                if match:
                    metadata["doc_id"] = match.group(1).strip()
                    break

        print(f"Обработано: {file_name} -> НАЗВАНИЕ: '{metadata['doc_name']}' | ID: {metadata['doc_id']}")
        return metadata


    loader = DirectoryLoader(
        folder_path,
        glob="**/*.md",
        show_progress=True,
        loader_kwargs={'autodetect_encoding': True},
        use_multithreading=True
    )

    docs = loader.load()

    # Добавляем метаданные к каждому документу
    for doc in docs:
        doc.metadata.update(parse_metadata(doc.page_content, doc.metadata['source']))
        # Для отладки
        print(f"Документ: {doc.metadata.get('doc_name', 'Без названия')} | ID: {doc.metadata.get('doc_id', 'N/A')}")

    print(f"\nИтого загружено {len(docs)} документов с метаданными")
    return docs


# Разделение на чанки (сохраняем метаданные)
def split_documents(docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=850,
        chunk_overlap=80,
        separators=["\n\n", "\n", " ", ""]
    )

    # Разделяем, сохраняя метаданные в каждом чанке
    chunks = []
    for doc in docs:
        chunks.extend(text_splitter.split_documents([doc]))

    print(f"Разделено на {len(chunks)} чанков с метаданными")
    return chunks

# Создание векторной базы (без изменений)
def create_vector_db(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="deepvk/USER-bge-m3")
    db = FAISS.from_documents(chunks, embeddings)
    return db

# Основной процесс
folder_path = '/content/create_index_base'

# 1. Загрузка документов с метаданными
documents = load_documents(folder_path)

# 2. Разделение на чанки (метаданные сохраняются)
chunks = split_documents(documents)

# 3. Создание и сохранение векторной базы
db = create_vector_db(chunks)
db.save_local("Rules_and_orders")

print("Векторная база с метаданными сохранена в 'Rules_and_orders'")


In [ ]:
# @title Создание векторной базы с метаданными ОБЪЕДИНЕНИЕ ФАЙЛОВ С МЕТА МЕТКАМИ
import os
import re
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Функция для парсинга метаданных
def parse_metadata(content, file_path):
    """Парсер метаданных с использованием имени файла как резервного варианта"""
    # Получаем имя файла без расширения
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    metadata = {
        "source": file_path,
        "doc_name": file_name,  # По умолчанию используем имя файла
        "doc_id": "N/A"
    }

    # Варианты поиска ID
    id_patterns = [
        r'ID:\s*`?([a-zA-Z0-9]+)`?',
        r'Идентификатор:\s*(\w+)',
        r'Document ID:\s*(\w+)'
    ]

    lines = [line.strip() for line in content.split('\n') if line.strip()]

    # Поиск названия в содержимом (если отличается от имени файла)
    name_patterns = [
        r'^#\s*Документ:\s*(.+?)\s*$',
        r'^#\s*(.+?)\s*\nID:',
        r'^#\s*(.+?)\s*$'
    ]

    for line in lines[:5]:
        for pattern in name_patterns:
            match = re.search(pattern, line, re.IGNORECASE)
            if match and match.group(1).strip().lower() != file_name.lower():
                metadata["doc_name"] = match.group(1).strip()
                break

    # Поиск ID
    for line in lines[:10]:
        for pattern in id_patterns:
            match = re.search(pattern, line, re.IGNORECASE)
            if match:
                metadata["doc_id"] = match.group(1).strip()
                break

    print(f"Обработано: {file_name} -> НАЗВАНИЕ: '{metadata['doc_name']}' | ID: {metadata['doc_id']}")
    return metadata

# Загрузка всех документов с метаданными
def load_documents(folder_path):
    loader = DirectoryLoader(
        folder_path,
        glob="**/*.txt",  # Загружаем все .txt файлы
        show_progress=True,
        use_multithreading=True,
        loader_kwargs={'autodetect_encoding': True}
    )

    md_loader = DirectoryLoader(
        folder_path,
        glob="**/*.md",  # Загружаем все .md файлы
        show_progress=True,
        use_multithreading=True,
        loader_kwargs={'autodetect_encoding': True}
    )

    # Объединяем загруженные документы
    docs = loader.load() + md_loader.load()

    # Добавляем метаданные к каждому документу
    for doc in docs:
        doc.metadata.update(parse_metadata(doc.page_content, doc.metadata['source']))
        # Для отладки
        print(f"Документ: {doc.metadata.get('doc_name', 'Без названия')} | ID: {doc.metadata.get('doc_id', 'N/A')}")

    print(f"\nИтого загружено {len(docs)} документов с метаданными")
    return docs

# Разделение на чанки с сохранением метаданных
def split_documents(docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=750,
        chunk_overlap=80,
        separators=["\n\n", "\n", " ", ""]
    )

    # Разделяем, сохраняя метаданные в каждом чанке
    chunks = []
    for doc in docs:
        current_chunks = text_splitter.split_documents([doc])
        chunks.extend(current_chunks)

    print(f"Разделено на {len(chunks)} чанков с метаданными")
    return chunks

# Создание векторной базы
def create_vector_db(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="deepvk/USER-bge-m3")
    db = FAISS.from_documents(chunks, embeddings)
    return db

# Основной процесс
folder_path = "/content/doc"  # Путь к папке с документами

# 1. Загрузка документов с метаданными
documents = load_documents(folder_path)

# 2. Разделение на чанки с сохранением метаданных
chunks = split_documents(documents)

# 3. Создание и сохранение векторной базы
db = create_vector_db(chunks)
db.save_local("data_base_faiss_with_metadata")

print("Векторная база с метаданными успешно создана и сохранена в папке 'data_base_faiss_with_metadata'")

0it [00:00, ?it/s]
100%|██████████| 10/10 [00:08<00:00,  1.19it/s]
/tmp/ipython-input-2-2576732502.py:105: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="deepvk/USER-bge-m3")


Обработано: Северо-Кавказская железная дорога -> НАЗВАНИЕ: 'Северо-Кавказская железная дорога' | ID: 0172d652
Документ: Северо-Кавказская железная дорога | ID: 0172d652
Обработано: Куйбышевская железная дорога -> НАЗВАНИЕ: 'Куйбышевская железная дорога' | ID: ad2d0c59
Документ: Куйбышевская железная дорога | ID: ad2d0c59
Обработано: Западно-Сибирская железная дорога -> НАЗВАНИЕ: 'Западно-Сибирская железная дорога' | ID: a43c5d77
Документ: Западно-Сибирская железная дорога | ID: a43c5d77
Обработано: Северная железная дорога -> НАЗВАНИЕ: 'Северная железная дорога' | ID: 5372e682
Документ: Северная железная дорога | ID: 5372e682
Обработано: Красноярская железная дорога -> НАЗВАНИЕ: 'Красноярская железная дорога' | ID: 98e26b78
Документ: Красноярская железная дорога | ID: 98e26b78
Обработано: Горьковская железная дорога -> НАЗВАНИЕ: 'Горьковская железная дорога' | ID: a80978b3
Документ: Горьковская железная дорога | ID: a80978b3
Обработано: Свердловская железная дорога -> НАЗВАНИЕ: 'Свердл

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

Векторная база с метаданными успешно создана и сохранена в папке 'data_base_faiss_with_metadata'


In [ ]:
# @title Модифицированный код 2 для работы с метаданными ОДИНОЧНЫЙ

from langchain_community.document_loaders import DirectoryLoader, TextLoader
#new_folder_name = '/content/create_index_base'
#os.makedirs(new_folder_name)

# Загрузка документов с парсингом метаданных
def load_documents(folder_path):
    def parse_metadata(content, file_path):
        """Парсер метаданных с использованием имени файла как резервного варианта"""
        # Получаем имя файла без расширения
        file_name = os.path.splitext(os.path.basename(file_path))[0]

        metadata = {
            "source": file_path,
            "doc_name": file_name,  # По умолчанию используем имя файла
            "doc_id": "N/A"
        }

        # Варианты поиска ID
        id_patterns = [
            r'ID:\s*`?([a-zA-Z0-9]+)`?',
            r'Идентификатор:\s*(\w+)',
            r'Document ID:\s*(\w+)'
        ]

        lines = [line.strip() for line in content.split('\n') if line.strip()]

        # Поиск названия в содержимом (если отличается от имени файла)
        name_patterns = [
            r'^#\s*Документ:\s*(.+?)\s*$',
            r'^#\s*(.+?)\s*\nID:',
            r'^#\s*(.+?)\s*$'
        ]

        for line in lines[:5]:
            for pattern in name_patterns:
                match = re.search(pattern, line, re.IGNORECASE)
                if match and match.group(1).strip().lower() != file_name.lower():
                    metadata["doc_name"] = match.group(1).strip()
                    break

        # Поиск ID
        for line in lines[:10]:
            for pattern in id_patterns:
                match = re.search(pattern, line, re.IGNORECASE)
                if match:
                    metadata["doc_id"] = match.group(1).strip()
                    break

        print(f"Обработано: {file_name} -> НАЗВАНИЕ: '{metadata['doc_name']}' | ID: {metadata['doc_id']}")
        return metadata


    loader = DirectoryLoader(
        folder_path,
        glob="**/*.md",
        show_progress=True,
        loader_kwargs={'autodetect_encoding': True},
        use_multithreading=True
    )

    docs = loader.load()

    # Добавляем метаданные к каждому документу
    for doc in docs:
        doc.metadata.update(parse_metadata(doc.page_content, doc.metadata['source']))
        # Для отладки
        print(f"Документ: {doc.metadata.get('doc_name', 'Без названия')} | ID: {doc.metadata.get('doc_id', 'N/A')}")

    print(f"\nИтого загружено {len(docs)} документов с метаданными")
    return docs


# Разделение на чанки (сохраняем метаданные)
from langchain.text_splitter import MarkdownHeaderTextSplitter

def split_documents(docs):
    # Сначала выделяем структурные элементы
    headers_to_split_on = [
        ("#", "Заголовок 1"),
        ("##", "Заголовок 2"),
        ("###", "Заголовок 3"),
    ]

    # Основной сплиттер по структуре
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on,
        return_each_line=False
    )

    # Финалный сплиттер для контроля размера
    final_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200,  # Увеличили размер для учета метаданных
        chunk_overlap=200,  # Увеличили перекрытие
        separators=["\n\n## ", "\n\n# ", "\n\n", "\n", " "],
        keep_separator=True
    )

    chunks = []
    for doc in docs:
        # Первичное разделение по заголовкам
        md_splits = markdown_splitter.split_text(doc.page_content)

        # Вторичное разделение по размеру
        for split in md_splits:
            # Сохраняем оригинальные метаданные + заголовки из разделения
            metadata = {**doc.metadata, **split.metadata}
            sub_chunks = final_splitter.split_documents([Document(
                page_content=split.page_content,
                metadata=metadata
            )])
            chunks.extend(sub_chunks)

    print(f"Разделено на {len(chunks)} чанков с улучшенной структурой")
    return chunks

# Создание векторной базы (без изменений)
def create_vector_db(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="deepvk/USER-bge-m3")
    db = FAISS.from_documents(chunks, embeddings)
    return db

# Основной процесс
folder_path = '/content/create_index_base'

# 1. Загрузка документов с метаданными
documents = load_documents(folder_path)

# 2. Разделение на чанки (метаданные сохраняются)
chunks = split_documents(documents)

# 3. Создание и сохранение векторной базы
db = create_vector_db(chunks)
db.save_local("Rules_and_orders")

print("Векторная база с метаданными сохранена в 'Rules_and_orders'")
import matplotlib.pyplot as plt

chunk_lengths = [len(chunk.page_content) for chunk in chunks]
plt.hist(chunk_lengths, bins=30)
plt.xlabel('Длина чанка')
plt.ylabel('Количество')
plt.show()


# inferens

In [2]:
# Глобальные переменные
llm = None
SYSTEM_PROMPT = ""
PROMPT_TEMPLATE = ""

llm = LLM(model="./YandexGPT-5-Lite-8B-instruct-Q4_K_M.gguf",
          tokenizer="yandex/YandexGPT-5-Lite-8B-pretrain",
          max_model_len = 25500
          )  # Используем строку с именем токенизатора

INFO 07-21 08:02:35 [config.py:823] This model supports multiple tasks: {'generate', 'score', 'reward', 'embed', 'classify'}. Defaulting to 'generate'.
ERROR 07-21 08:02:35 [config.py:114] Error retrieving safetensors: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: './YandexGPT-5-Lite-8B-instruct-Q4_K_M.gguf'., retrying 1 of 2
ERROR 07-21 08:02:37 [config.py:112] Error retrieving safetensors: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: './YandexGPT-5-Lite-8B-instruct-Q4_K_M.gguf'.
INFO 07-21 08:02:37 [config.py:3268] Downcasting torch.float32 to torch.float16.
WARNING 07-21 08:02:38 [config.py:931] gguf quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 07-21 08:02:38 [arg_utils.py:1642] Compute Capability < 8.0 is not supported by the V1 Engine.

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 07-21 08:02:39 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.1) with config: model='./YandexGPT-5-Lite-8B-instruct-Q4_K_M.gguf', speculative_config=None, tokenizer='yandex/YandexGPT-5-Lite-8B-pretrain', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=25500, download_dir=None, load_format=LoadFormat.GGUF, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=gguf, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, otlp_traces_endpoint=None, collect_detailed_traces=None), seed=0, served_model_name=./YandexGPT-5-Lite-8B-instruct-Q4_K_M.gguf, num_scheduler_steps=1,

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 07-21 08:10:59 [model_runner.py:1671] Graph capturing finished in 148 secs, took 0.31 GiB
INFO 07-21 08:10:59 [llm_engine.py:428] init engine (profile, create kv cache, warmup model) took 433.50 seconds


In [ ]:
# @title локальный инференс
from typing import Dict
# Глобальная конфигурация
SYSTEM_PROMPT = """Ты — помощник по работе с документами в ОАО РЖД. Отвечай строго по документам.
Формат ответа:
* Основной ответ - развернуто по запросу

"""

PROMPT_TEMPLATE = """Документ: {document_name}
Контекст:
{context}

Вопрос: {query}

Ответь максимально точно, используя только указанный контекст."""


class RAGSystem:
    def __init__(self):
        # Проверка инициализации глобальной модели
        if llm is None:
            raise ValueError("Глобальная переменная llm не инициализирована!")

        # Настройка логирования
        logging.getLogger("vllm").setLevel(logging.WARNING)

        # Инициализация параметров генерации
        self.sampling_params = SamplingParams(
            temperature=0.3,
            top_p=0.9,
            max_tokens=2048
        )

        # Конфигурация баз данных
        self.databases_config = {
            "1": {
                "path": "/content/drive/MyDrive/meta_base_faiss/project_BZ/overall",
                #"path": "/content/Rules_and_orders", # для тестов
                "name": "Общая",
                "doc_prefix": "БЗ май 2025"
            },
            "2": {
                "path": "/content/drive/MyDrive/meta_base_faiss/project_BZ/SVO",
                "name": "СВО",
                "doc_prefix": "БЗ май 2025"
            },
            "3": {
                "path": "/content/drive/MyDrive/meta_base_faiss/project_BZ/ChS",
                "name": "Санкции+Женщины+ЧС+Сервисный портал",
                "doc_prefix": "БЗ май 2025"
            }
        }

        # Загрузка баз данных
        self.vector_databases = self.load_databases()

    def load_databases(self) -> Dict:
        """Загружает FAISS индексы в память с поддержкой метаданных"""
        embeddings = HuggingFaceEmbeddings(model_name="deepvk/USER-bge-m3")

        """ не заработал хотел чтобы меньше использовала GPU """
        #embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        loaded_dbs = {}

        for db_id, config in self.databases_config.items():
            try:
                loaded_dbs[db_id] = {
                    "instance": FAISS.load_local(
                        folder_path=config["path"],
                        embeddings=embeddings,
                        index_name="index",
                        allow_dangerous_deserialization=True
                    ),
                    "name": config["name"],
                    "prefix": config["doc_prefix"]
                }
                print(f"✅ База {db_id}: {config['name']} (префикс: {config['doc_prefix']}) загружена")
            except Exception as e:
                print(f"❌ Ошибка загрузки базы {db_id}: {e}")
                raise

        return loaded_dbs

    def _print_database_menu(self):
        """Выводит меню выбора базы знаний"""
        print("\n" + "="*50)
        print("Выберите базу знаний:")
        for num, db in self.vector_databases.items():
            print(f"[{num}] - {db['name']}")

    def _generate_response(self, prompt: str):
        """Универсальный метод генерации ответа с обработкой ошибок"""
        try:
            # Для vLLM версии 0.4.2+
            if hasattr(llm, 'generate'):
                outputs = llm.generate(prompt, self.sampling_params)
                if hasattr(outputs, 'outputs'):  # Новый формат vLLM
                    return outputs.outputs[0].text
                elif isinstance(outputs, list):  # Старый формат vLLM
                    return outputs[0].outputs[0].text
                return outputs.text.split('\n')[-1]  # Фолбэк обработка

            # Для других версий/интерфейсов
            result = llm(prompt, sampling_params=self.sampling_params)
            return result.text if hasattr(result, 'text') else str(result)

        except Exception as e:
            print(f"Ошибка генерации: {str(e)[:200]}")  # Обрезаем длинные сообщения
            return None

    def _process_query(self, query: str, db_context: Dict):
        """Оптимизированная обработка запроса с метаданными"""
        # Получаем релевантные документы
        docs = db_context["instance"].similarity_search(query, k=5)

        # Извлекаем контент и метаданные
        context_parts = []
        metadata = {
            'sources': set(),
            'sections': set(), # подумать как маркировать разделы
            'doc_ids': set()
        }

        for doc in docs:
            context_parts.append(doc.page_content)

            # Извлекаем метаданные
            if hasattr(doc, 'metadata'):
                meta = doc.metadata
                if 'doc_name' in meta: metadata['sources'].add(meta['doc_name'])
                if 'doc_id' in meta: metadata['doc_ids'].add(meta['doc_id'])
                if 'section' in meta: metadata['sections'].add(meta['section'])

                # Альтернативные варианты имен полей
                header = re.search(r'^#+\s*(.+?)\n', doc.page_content)
                if header: metadata['sections'].add(header.group(1).strip())

        # Формируем промпт
        system_part = SYSTEM_PROMPT.format(
            document_name=db_context['name'],
            document_prefix=db_context.get('prefix', '')
        ).strip()

        user_part = PROMPT_TEMPLATE.format(
            document_name=db_context['name'],
            document_prefix=db_context.get('prefix', ''),
            context='\n\n'.join([f"📄 Документ {i+1}:\n{text}" for i, text in enumerate(context_parts)]),
            query=query
        ).strip()

        full_prompt = f"{system_part}\n\n{user_part}"

        # Генерируем ответ
        response = self._generate_response(full_prompt)

        # Форматируем вывод
        if response is None:
            response = "Не удалось сгенерировать ответ"

        # Подготовка метаданных для отображения
        meta_lines = [
            f"🔹 📋 Источники: {', '.join(metadata['sources'])}",
            #f"🔹 {db_context['name']} ({db_context.get('prefix', '')})", # если база из нескольких документов то берется только первый
            #f"🔹 📋 Источники: {', '.join(metadata['sources'])} ({db_context.get('prefix', '')})", # префикс добавляется от базы а если несколько документов в базе?
            #f"📂 Разделы: {', '.join(metadata['sections']) or 'не указаны'}" # подумать как размечать разделы
        ]
        # пока заблокировал из-за путаницы в конце ответа (слишком много источников указано)

        #if metadata['sources']:
        #   meta_lines.append(f"📋 Источники: {', '.join(metadata['sources'])}")
        #if metadata['doc_ids']:
        #    meta_lines.append(f"🆔 ID документов: {', '.join(metadata['doc_ids'])}")

        return f"{response}\n\n" + '\n'.join(meta_lines)
        #return f"{response}\n\n"



    def run(self):
        """Основной диалоговый интерфейс"""
        print("\n🚂 Добро пожаловать в RAG-систему РЖД!")
        current_db = None

        while True:
            if not current_db:
                self._print_database_menu()
                choice = input("\nНомер базы (1-3) или 'выход':\n ").strip()

                if choice.lower() in ('выход', 'exit'):
                    break

                if choice not in self.vector_databases:
                    print("⚠️ Некорректный ввод!")
                    continue

                current_db = self.vector_databases[choice]
                print(f"\n🔍 Активная база: {current_db['name']}")

            query = input("\n💬 Ваш вопрос ('смена'/'выход'):\n ").strip()

            if query.lower() == 'выход':
                break
            if query.lower() == 'смена':
                current_db = None
                continue

            response_text = self._process_query(query, current_db)

            print("\n" + "🔷" * 25)
            print(textwrap.fill(response_text, width=160))
            print("🔷" * 25)

        print("\n" + "🔷" * 10)
        print("     До новых встреч!")
        print("🔷" * 10)


if __name__ == "__main__":
    rag_system = RAGSystem()
    rag_system.run()

In [14]:
# @title очистка памяти
import torch
import gc
from psutil import virtual_memory

def clear_memory(verbose=True):
    """Полная очистка памяти (GPU и RAM)"""
    # Проверка доступности CUDA
    cuda_available = torch.cuda.is_available()

    if verbose:
        # Вывод информации о памяти перед очисткой
        print("═"*40)
        print("Состояние памяти до очистки:")
        if cuda_available:
            print(f"GPU память:")
            print(f"Использовано: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
            print(f"Зарезервировано: {torch.cuda.memory_reserved()/1024**2:.2f} MB")

        ram = virtual_memory()
        print(f"\nRAM память:")
        print(f"Использовано: {ram.used/1024**3:.2f} GB")
        print(f"Доступно: {ram.available/1024**3:.2f} GB")
        print("═"*40 + "\n")

    # Очистка GPU
    if cuda_available:
        torch.cuda.empty_cache()

    # Очистка оперативной памяти
    gc.collect()

    if verbose:
        print("Выполняется очистка памяти...\n")

        if cuda_available:
            # Принудительная очистка с помощью torch
            with torch.no_grad():
                torch.cuda.synchronize()

            print("После очистки:")
            print(f"GPU память:")
            print(f"Использовано: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
            print(f"Зарезервировано: {torch.cuda.memory_reserved()/1024**2:.2f} MB")

            ram = virtual_memory()
            print(f"\nRAM память:")
            print(f"Использовано: {ram.used/1024**3:.2f} GB")
            print(f"Доступно: {ram.available/1024**3:.2f} GB")
        else:
            print("GPU память не доступна")

        print("\n✅ Очистка памяти завершена!")

# Использование
clear_memory()


════════════════════════════════════════
Состояние памяти до очистки:
GPU память:
Использовано: 14462.89 MB
Зарезервировано: 14642.00 MB

RAM память:
Использовано: 4.94 GB
Доступно: 3.40 GB
════════════════════════════════════════

Выполняется очистка памяти...

После очистки:
GPU память:
Использовано: 11723.48 MB
Зарезервировано: 14642.00 MB

RAM память:
Использовано: 4.94 GB
Доступно: 3.40 GB

✅ Очистка памяти завершена!


# Inferens c GRADIO

In [ ]:
!pip install pyngrok # не обязательно в версии gradio 3+

In [ ]:
# @title рабочая сокращенная версия
import gradio as gr
from typing import Dict
import textwrap
import logging
import re

# Глобальная конфигурация
SYSTEM_PROMPT = """Ты — помощник по работе с документами в ОАО РЖД. Отвечай строго по документам.
Формат ответа:
* Основной ответ - развернуто по запросу
"""

PROMPT_TEMPLATE = """Документ: {document_name} ({document_prefix})
Контекст:
{context}

Вопрос: {query}

Ответь максимально точно, используя только указанный контекст."""


class RAGSystem:
    def __init__(self, gradio_mode=False):
        self.gradio_mode = gradio_mode
        # Проверка инициализации глобальной модели
        if llm is None:
            raise ValueError("Глобальная переменная llm не инициализирована!")

        # Настройка логирования
        logging.getLogger("vllm").setLevel(logging.WARNING)

        # Инициализация параметров генерации
        self.sampling_params = SamplingParams(
            temperature=0.3,
            top_p=0.9,
            max_tokens=512  # Уменьшаем max_tokens
        )

        # Конфигурация баз данных
        self.databases_config = {
            "1": {
                "path": "/content/drive/MyDrive/meta_base_faiss/Rules_and_orders",
                "name": "Правила технической эксплуатации железных дорог в РФ",
                "doc_prefix": "Приказ №250 от 2022"
            },
            "2": {
                "path": "/content/drive/MyDrive/meta_base_faiss/OT",
                "name": "СИСТЕМА УПРАВЛЕНИЯ ОХРАНОЙ ТРУДА В ОАО «РЖД»",
                "doc_prefix": "СТО РЖД 15.013-2021"
            },
            "3": {
                "path": "/content/drive/MyDrive/meta_base_faiss/Conducting",
                "name": "ИНСТРУКЦИЯ по делопроизводству и документированию управленческой деятельности в ОАО «РЖД»",
                "doc_prefix": "Приказ ОАО «РЖД» от 14.12.2017 г. № 120"
            }
        }

        # Загрузка баз данных
        self.vector_databases = self.load_databases()
        self.current_db = None

    def load_databases(self) -> Dict:
        """Загружает FAISS индексы в память с поддержкой метаданных"""
        embeddings = HuggingFaceEmbeddings(model_name="deepvk/USER-bge-m3")
        loaded_dbs = {}

        for db_id, config in self.databases_config.items():
            try:
                loaded_dbs[db_id] = {
                    "instance": FAISS.load_local(
                        folder_path=config["path"],
                        embeddings=embeddings,
                        index_name="index",
                        allow_dangerous_deserialization=True
                    ),
                    "name": config["name"],
                    "prefix": config["doc_prefix"]
                }
                if not self.gradio_mode:
                    print(f"✅ База {db_id}: {config['name']} (префикс: {config['doc_prefix']}) загружена")
            except Exception as e:
                if not self.gradio_mode:
                    print(f"❌ Ошибка загрузки базы {db_id}: {e}")
                raise

        return loaded_dbs

    def _generate_response(self, prompt: str):
        """Универсальный метод генерации ответа с обработкой ошибок"""
        try:
            if hasattr(llm, 'generate'):
                outputs = llm.generate(prompt, self.sampling_params)
                if hasattr(outputs, 'outputs'):
                    return outputs.outputs[0].text
                elif isinstance(outputs, list):
                    return outputs[0].outputs[0].text
                return outputs.text.split('\n')[-1]

            result = llm(prompt, sampling_params=self.sampling_params)
            return result.text if hasattr(result, 'text') else str(result)

        except Exception as e:
            error_msg = f"Ошибка генерации: {str(e)[:200]}"
            if not self.gradio_mode:
                print(error_msg)
            return error_msg

    def _process_query(self, query: str, db_context: Dict):
        """Оптимизированная обработка запроса с метаданными"""
        docs = db_context["instance"].similarity_search(query, k=3)  # Уменьшаем k

        context_parts = []
        metadata = {'sources': set(), 'sections': set(), 'doc_ids': set()}

        for doc in docs:
            context_parts.append(doc.page_content)

            if hasattr(doc, 'metadata'):
                meta = doc.metadata
                if 'doc_name' in meta:
                    metadata['sources'].add(meta['doc_name'])
                if 'doc_id' in meta:
                    metadata['doc_ids'].add(meta['doc_id'])
                if 'section' in meta:
                    metadata['sections'].add(meta['section'])

                header = re.search(r'^#+\s*(.+?)\n', doc.page_content)
                if header:
                    metadata['sections'].add(header.group(1).strip())

        system_part = SYSTEM_PROMPT.format(
            document_name=db_context['name'],
            document_prefix=db_context.get('prefix', '')
        ).strip()

        user_part = PROMPT_TEMPLATE.format(
            document_name=db_context['name'],
            document_prefix=db_context.get('prefix', ''),
            context='\n\n'.join([f"📄 Документ {i + 1}:\n{text}" for i, text in enumerate(context_parts)]),
            query=query
        ).strip()

        full_prompt = f"{system_part}\n\n{user_part}"
        response = self._generate_response(full_prompt) or "Не удалось сгенерировать ответ"

        meta_lines = [f"🔹 📋 Источники: {', '.join(metadata['sources'])}"]
        return f"{response}\n\n" + '\n'.join(meta_lines)

    def gradio_interface(self, query: str, database: str):
        """Обработчик для Gradio интерфейса"""
        if database not in self.vector_databases:
            return "⚠️ Выберите корректную базу данных"

        db_context = self.vector_databases[database]
        return self._process_query(query, db_context)

    def run_console(self):
        """Консольный интерфейс"""

    def run_gradio(self):
        """Запуск Gradio (совместимость с версией 3.41.0)"""
        import gradio as gr

        with gr.Blocks(title="RAG-система РЖД", theme=gr.themes.Soft()) as demo:
            gr.Markdown("# 🚂 RAG-система ОАО РЖД")

            # 1. Простые компоненты без сложной логики состояний
            database = gr.Dropdown(
                choices=["1", "2", "3"],
                label="База знаний",
                value="1"
            )
            query = gr.Textbox(label="Ваш вопрос", placeholder="Введите запрос...", lines=3)
            submit_btn = gr.Button("Отправить", variant="primary")
            output = gr.Textbox(label="Ответ системы", interactive=False, lines=10)

            # 2. Обработчик запроса (с отловом ошибок)
            def process_query(query_text, db_id):
                try:
                    if not query_text.strip():
                        return "❌ Введите вопрос!"

                    if db_id not in self.vector_databases:
                        return f"❌ База {db_id} не найдена!"

                    db = self.vector_databases[db_id]
                    answer = self._process_query(query_text, db)

                    return answer if answer else "⚠️ Система не вернула ответ"

                except Exception as e:
                    error_msg = f"⛔ Ошибка: {str(e)[:200]}"
                    print("Debug:", error_msg)  # Логируем в консоль
                    return error_msg

            # 3. Подключаем кнопку (без api_name)
            submit_btn.click(
                process_query,
                inputs=[query, database],
                outputs=output
            )

            # 4. Запуск без enable_queue (совместимость с Gradio 3.x)
        demo.launch(
            share=True,
            debug=True,  # Включаем отладку
            #server_port=7861  # Меняем порт, если 7860 занят
        )
if __name__ == "__main__":
    # Тестируем только Gradio
    rag_system = RAGSystem()
    rag_system.run_gradio()

In [3]:
# @title рабочая красивая версия
import gradio as gr
from typing import Dict
import textwrap
import logging
import re

# Глобальная конфигурация
SYSTEM_PROMPT = """Ты — помощник по работе с документами в ОАО РЖД. Отвечай строго по документам.
Формат ответа:
* Основной ответ - развернуто по запросу
"""

PROMPT_TEMPLATE = """Документ: {document_name}
Контекст:
{context}

Вопрос: {query}

Ответь максимально развернуто и точно, используя только указанный контекст."""


class RAGSystem:
    def __init__(self, gradio_mode=False):
        self.gradio_mode = gradio_mode
        # Проверка инициализации глобальной модели
        if llm is None:
            raise ValueError("Глобальная переменная llm не инициализирована!")

        # Настройка логирования
        logging.getLogger("vllm").setLevel(logging.WARNING)

        # Инициализация параметров генерации
        self.sampling_params = SamplingParams(
            temperature=0.3,
            top_p=0.9,
            max_tokens=1048  # Уменьшаем max_tokens
        )
        # Конфигурация баз данных 2 базы
        self.databases_config = {
            "1": {
                "path": "/content/United",
                "name": "Все вопросы",
                "doc_prefix": "БЗ май 2025"
            },

            "2": {
                "path": "/content/contacts",
                "name": "Адресная книга ФО по дорогам",
                "doc_prefix": "Контакты СКЦ"
            }
          }
        # Загрузка баз данных
        self.vector_databases = self.load_databases()
        self.current_db = None

    def load_databases(self) -> Dict:
        """Загружает FAISS индексы в память с поддержкой метаданных"""
        embeddings = HuggingFaceEmbeddings(model_name="deepvk/USER-bge-m3")
        loaded_dbs = {}

        for db_id, config in self.databases_config.items():
            try:
                loaded_dbs[db_id] = {
                    "instance": FAISS.load_local(
                        folder_path=config["path"],
                        embeddings=embeddings,
                        index_name="index",
                        allow_dangerous_deserialization=True
                    ),
                    "name": config["name"],
                    "prefix": config["doc_prefix"]
                }
                if not self.gradio_mode:
                    print(f"✅ База {db_id}: {config['name']} (префикс: {config['doc_prefix']}) загружена")
            except Exception as e:
                if not self.gradio_mode:
                    print(f"❌ Ошибка загрузки базы {db_id}: {e}")
                raise

        return loaded_dbs

    def _generate_response(self, prompt: str):
        """Универсальный метод генерации ответа с обработкой ошибок"""
        try:
            if hasattr(llm, 'generate'):
                outputs = llm.generate(prompt, self.sampling_params)
                if hasattr(outputs, 'outputs'):
                    return outputs.outputs[0].text
                elif isinstance(outputs, list):
                    return outputs[0].outputs[0].text
                return outputs.text.split('\n')[-1]

            result = llm(prompt, sampling_params=self.sampling_params)
            return result.text if hasattr(result, 'text') else str(result)

        except Exception as e:
            error_msg = f"Ошибка генерации: {str(e)[:200]}"
            if not self.gradio_mode:
                print(error_msg)
            return error_msg

    def _process_query(self, query: str, db_context: Dict):
        """Оптимизированная обработка запроса с метаданными"""
        docs = db_context["instance"].similarity_search(query, k=3)  # Уменьшаем k

        context_parts = []
        metadata = {'sources': set(), 'sections': set(), 'doc_ids': set()}

        for doc in docs:
            context_parts.append(doc.page_content)

            if hasattr(doc, 'metadata'):
                meta = doc.metadata
                if 'doc_name' in meta:
                    metadata['sources'].add(meta['doc_name'])
                if 'doc_id' in meta:
                    metadata['doc_ids'].add(meta['doc_id'])
                if 'section' in meta:
                    metadata['sections'].add(meta['section'])

                header = re.search(r'^#+\s*(.+?)\n', doc.page_content)
                if header:
                    metadata['sections'].add(header.group(1).strip())

        system_part = SYSTEM_PROMPT.format(
            document_name=db_context['name'],
            document_prefix=db_context.get('prefix', '')
        ).strip()

        user_part = PROMPT_TEMPLATE.format(
            document_name=db_context['name'],
            document_prefix=db_context.get('prefix', ''),
            context='\n\n'.join([f"📄 Документ {i + 1}:\n{text}" for i, text in enumerate(context_parts)]),
            query=query
        ).strip()

        full_prompt = f"{system_part}\n\n{user_part}"
        response = self._generate_response(full_prompt) or "Не удалось сгенерировать ответ"

        meta_lines = [f"🔹 📋 Источники: {', '.join(metadata['sources'])}"]
        return f"{response}\n\n" + '\n'.join(meta_lines)

    def gradio_interface(self, query: str, database: str):
        """Обработчик для Gradio интерфейса"""
        if database not in self.vector_databases:
            return "⚠️ Выберите корректную базу данных"

        db_context = self.vector_databases[database]
        return self._process_query(query, db_context)

    def run_console(self):
        """Консольный интерфейс"""

    def run_gradio(self):
        """Запуск Gradio интерфейса с отдельным выбором базы"""
        with gr.Blocks(title="RAG-система РЖД", theme=gr.themes.Soft()) as demo:
            gr.Markdown("# 🚂 RAG-система ОАО РЖД")
            gr.Markdown("### Проект 'ЦБР' Система 'Вопрос-Ответ' ")

            # Состояние для хранения выбранной базы
            current_db = gr.State("1")

            with gr.Row():
                with gr.Column(scale=1):
                    # Выбор базы знаний с описанием
                    database = gr.Dropdown(
                        choices=[
                            ("1 - Общая база вопросов", "1"),
                            ("2 - Адресная книга", "2")
                        ],
                        label="База знаний",
                        value="1"
                    )
                    gr.Markdown("**Описание баз:**")
                    gr.Markdown("""
                    - 1: Содержит ответы на вопросы по темам: "Общие", "ЧС", "Женщины", "Санкции", "СВО", Сервисный портал.
                    - 2: Содержит контактные данные и адреса СКЦ.
                    - Примеры вопросов в нижней части экрана (каждый пример активный)
                    """)

                with gr.Column(scale=3):
                    # Поле вопроса и ответа
                    query = gr.Textbox(
                        label="Ваш вопрос",
                        placeholder="Введите вопрос ...",
                        lines=3
                    )
                    gr.Markdown("**Не забудьте проверить корректность выбранной базы**")  # Добавляем подсказку

                    submit_btn = gr.Button("Отправить", variant="primary")
                    output = gr.Textbox(
                        label="Ответ системы",
                        interactive=False,
                        lines=10,
                        max_lines=20
                    )

            # Обработчики
            def update_db(db_id):
                """Обновляет выбранную базу и возвращает информационное сообщение"""
                db_info = self.vector_databases[db_id]
                return (
                    db_id,  # Сохраняем в состояние
                    f"Активная база: {db_info['name']} ({db_info['prefix']})"
                )

            def process_query(query_text, db_id):
                """Обрабатывает запрос к выбранной базе"""
                if not query_text.strip():
                    return "❗ Введите вопрос"

                db_context = self.vector_databases[db_id]
                return self._process_query(query_text, db_context)

            # Привязка событий
            database.change(
                fn=update_db,
                inputs=[database],
                outputs=[current_db, output]
            )

            submit_btn.click(
                fn=process_query,
                inputs=[query, current_db],
                outputs=output
            )
            # разделение примеров для баз (new)
            with gr.Row():
                with gr.Column(scale=1):

                    # Примеры запросов для первой базы
                    with gr.Tab("Примеры для Общей базы вопросов"):
                        gr.Examples(
                            examples=[
                                ["По какой форме транспортного требования при проезде в поезде пригородного сообщения разрешается выход на станции, отличной от указанной в билете?"],
                                ["Круг вопросов (полномочий) билетных бюро?"],
                                ["Проезд работников ОАО «РЖД» и иных лиц в Крым?"],
                                ["Что подразумевается под «Чрезвычайной ситуацией»?"],
                                ["Расскажите об обучении женщин-машинистов в Новосибирске"],
                                ["Смогу ли я пользоваться своей зарплатной картой за рубежом?"],
                                ["Как действовать, если пришла повестка?"],
                                ["При регистрации на Сервисном портале заблокировался логин?"],
                                ["Не открывается мобильное приложение?"]
                            ],
                            inputs=[query],
                            label="Общие вопросы"
                        )

                    # Примеры запросов для второй базы
                    with gr.Tab("Примеры для Адресной книги"):
                        gr.Examples(
                            examples=[
                                ["Санкт-Петербург-Центральный"],
                                ["Московско-Курский"],
                                ["Москва"],
                                ["Как позвонить на Савеловский вокзал"],
                                ["Адрес офиса в Нижнем Новогороде"],
                                ["Курган"],
                            ],
                            inputs=[query],
                            label="Контакты"
                        )


            # 4. Запуск без enable_queue (совместимость с Gradio 3.x)
        demo.launch(
            share=True,
            debug=True,  # Включаем отладку
        )
if __name__ == "__main__":
    # Тестируем только Gradio
    rag_system = RAGSystem()
    rag_system.run_gradio()

/tmp/ipython-input-3-3062188386.py:59: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="deepvk/USER-bge-m3")


✅ База 1: Все вопросы (префикс: БЗ май 2025) загружена
✅ База 2: Адресная книга ФО по дорогам (префикс: Контакты СКЦ) загружена
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d16d1020c5bf4e6e5e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d16d1020c5bf4e6e5e.gradio.live
